# Поиск токсичных комментариев (с NLTK)

<h2> (Тема №13: Машинное обучение для текстов) <a class="tocSkip"> </h2>

<a name="1"></a>
## 1. Содержание

[1. Содержание](#1)

[2. Описание проекта](#2)

*    [2.1. Цель проекта](#21)
*    [2.2. Задачи проекта](#22)
*    [2.3. Описание данных](#23)
*    [2.4. План работы](#24)

[3. Подготовка данных](#3)

*    [3.1. Изучение данных](#31)
*    [3.2. Очистка данных](#32)
*    [3.3. Лемматизация `WordNetLemmatizer`](#33)
*    [3.4. Лемматизация `WordNetLemmatizer` с *POS*-тегами](#34)
*    [3.5. Вывод](#35)

[4. Обучение моделей](#4)

*    [4.1. Разделение данных на выборки](#41)
*    [4.2. `LogisticRegression`](#42)

    *    [4.2.1. `model_lr`](#421)
    *    [4.2.2. `model_lr_pos`](#422)
    
*    [4.3. `DecisionTreeClassifier`](#43)

    *    [4.3.1. `model_dt`](#431)
    *    [4.3.2. `model_dt_pos`](#432)

*    [4.4. `LGBMClassifier`](#44)

    *    [4.4.1. `model_lgbm50`](#441)
    *    [4.4.2. `model_lgbm50_pos`](#442)
    *    [4.4.3. `model_lgbm500`](#443)
    *    [4.4.4. `model_lgbm500_pos`](#444)

*    [4.5. Сравнение моделей](#45)
*    [4.6. Вывод](#46)

[5. Тестирование лучшей модели](#5)

*    [5.1. Качество модели](#51)
*    [5.2. Вывод](#52)

[6. Общий вывод](#6)

<a name="2"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
## 2. Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

<a name="21"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 2.1. Цель проекта

Провести исследование с целью построения модели машинного обучения, которая поможет классифицировать комментарии на позитивные и негативные.

Результаты исследования позволят магазину искать токсичные комментарии и отправлять их на модерацию.

<a name="22"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 2.2. Задачи проекта

Решим поставленную в проекте задачу **с помощью библиотеки *NLTK***.

1. Изучить данные.
2. Подготовить данные.
3. Лемматизировать данные.
4. Построить и обучить модели.
5. Протестировать лучшую модель.
6. Написать общий вывод.

Построим модель со значением метрики качества *F1* не меньше 0.75.

<a name="23"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 2.3. Описание данных

В нашем распоряжении набор данных с разметкой о токсичности правок.

Данные находятся в файле `toxic_comments.csv`.

Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак.

<a name="24"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 2.4. План работы

1. Изучим данные.
2. Очистим комментарии **с помощью библиотеки *NLTK***: переведём буквы в нижний регистр, оставим только латиницу, удалим стоп-слова. 
3. Лемматизируем комментарии без учёта части речи (без *POS*-тегов) **с помощью библиотеки *NLTK***.
4. Лемматизируем комментарии с учётом части речи (с *POS*-тегами) **с помощью библиотеки *NLTK***.
5. Разделим данные на обучающую и тестовую выборки для данных без *POS*-тегов и с *POS*-тегами.
6. Рассчитаем величину *TF-IDF* для данных без *POS*-тегов и с *POS*-тегами.
7. Обучим восемь моделей (`LogisticRegression`, `DecisionTreeClassifier` и `LGBMClassifier` с количеством деревьев `n_estimators`=50 и `n_estimators`=500) для данных без *POS*-тегов и с *POS*-тегами.
8. Сравненим модели.
9. Протестируем лучшую модель и напишем вывод.

<a name="3"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
## 3. Подготовка данных

In [1]:
import datetime
import nltk
import numpy as np
import pandas as pd
import re

from IPython.display import display
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords as nltk_stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

RANDOM_STATE = 12345
TEST_SIZE = 0.2

<a name="31"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 3.1. Изучение данных

Загрузим данные.

In [2]:
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    try:
        data = pd.read_csv(r'C:/Users/lorad/OneDrive/Documents/Моя папка/Data Science/Курсы/'
                            'Яндекс.Практикум. Специалист по Data Science/Проектная работа/'
                            '13. Машинное обучение для текстов/toxic_comments.csv')
    except:
        try:
            data = pd.read_csv(r'D:/Юлия/Data Science/Курсы/'
                                'Яндекс.Практикум. Специалист по Data Science/Проектная работа/'
                                '13. Машинное обучение для текстов/toxic_comments.csv')
        except:
            data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/'
            '2PACX-1vQ59JmNL2DruMdFkZoOga-GFUBVFTSgDnVt4Pt7SErYdQQ7hHrTSzRaBHYMhpwa_K4xlnKs_8zrW6di/'
                               'pub?gid=1802044232&single=true&output=csv')

In [3]:
display(data.sample(5))

,Unnamed: 0,text,toxic
153116,153273,That's the one about someone blowing up her ki...,0
158581,158740,Just so you don't think I'm going behind your ...,0
27513,27550,|decline=Indeed. :==( o ),0
58027,58091,Quit vandalizing the pages or you will be bloc...,0
158010,158169,Citation one through four also imo need removi...,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Датасет имеет большой размер.

In [5]:
print(data.shape)

(159292, 3)


In [6]:
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


Проверим наличие явных дубликатов.

In [7]:
data.duplicated().sum()

0

Подсчитаем количество классов в таргете.

In [8]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Наблюдается сильный дисбаланс классов в таргете.

<a name="32"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 3.2. Очистка данных

Мы будем решать поставленную в проекте задачу **с помощью библиотеки *NLTK*** (англ. *Natural Language Toolkit*, «инструментарий естественного языка») — ведущая платформа для создания NLP-программ на Python.

Напишем функцию `clear_text()`, которая очищает комментарии: переводит буквы в нижний регистр, оставляет только латиницу, удаляет стоп-слова (`stop_words`).

In [9]:
def clear_text(text):
    stop_words = set(nltk_stopwords.words('english'))
    # переведём в нижний регистр
    text = text.lower()
    # оставим только латиницу
    word_list = re.sub(r"[^a-z ]", ' ', text).split()
    # удалим stop_words
    word_notstop_list = [w for w in word_list if not w in stop_words]
    
    return ' '.join(word_notstop_list)

Добавим в датасет новый признак `clean_text` с очищенными комментариями.

In [10]:
%%time

data['clean_text'] = data['text'].apply(clear_text)

CPU times: total: 36.2 s
Wall time: 36.3 s


Выведем полученный датасет `data` с новым признаком `clean_text`.

In [11]:
display(data.sample(5))

,Unnamed: 0,text,toxic,clean_text
117340,117439,REDIRECT Talk:Absolutely (sketch show),0,redirect talk absolutely sketch show
108857,108954,"Menopause and spotting \n\nHi Invertzoo. ,\nAs...",0,menopause spotting hi invertzoo stated woman s...
120663,120768,Silver lining \n\nGo to and see if your brows...,0,silver lining go see browser passes test ie ff...
23468,23488,"""::::""""Projectively extended real numbers"""" is...",0,projectively extended real numbers fine chance...
111902,111999,Please cease adding unsourced information; you...,0,please cease adding unsourced information bloc...


<a name="33"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 3.3. Лемматизация `WordNetLemmatizer`

*Wordnet* – это большая, свободно распространяемая и общедоступная лексическая база данных для английского языка с целью установления структурированных семантических отношений между словами. Для того, чтобы лемматизировать комментарии, нужно создать экземпляр `WordNetLemmatizer` и вызвать функцию `lemmatize()` для одного слова.

Лемматизируем комментарии без учёта части речи (без *POS*-тегов).

In [12]:
def lemm_text(text):
    # создадим объект класса для лемматизации
    lemmatizer = WordNetLemmatizer()
    word_list = text.split()
    lemmatized_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    
    return lemmatized_text

Добавим в датасет новый признак `wnl_text` с комментариями, лемматизированными без *POS*-тегов.

In [13]:
%%time

beg_time = datetime.datetime.now()
data['wnl_text'] = data['clean_text'].apply(lemm_text)
data_lemm_time = (datetime.datetime.now()-beg_time).seconds

print(f'время выполнения лемматизации: {data_lemm_time} s')

время выполнения лемматизации: 18 s
CPU times: total: 18.1 s
Wall time: 18.3 s


<a name="34"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 3.4. Лемматизация `WordNetLemmatizer` с *POS*-тегами

Лемматизируем комментарии с учётом части речи (с *POS*-тегами) **с помощью библиотеки *NLTK***.

In [14]:
def get_wordnet_pos(word):
    # сопоставим POS-тег с первым символом, 
    # который принимает функция lemmatize()
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN)

In [15]:
def postag_lemm_text(text):
    # создадим объект класса для лемматизации с учетом POS-тегов
    lemmatizer = WordNetLemmatizer()
    word_list = text.split()
    lemmatized_text = ' '.join([
        lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list
    ])
    
    return lemmatized_text

Добавим в датасет новый признак `wnlpostag_text` с комментариями, лемматизированными с *POS*-тегами.

In [16]:
%%time

beg_time = datetime.datetime.now()
data['wnlpostag_text'] = data['clean_text'].apply(postag_lemm_text)
data_lemm_pos_time = (datetime.datetime.now()-beg_time).seconds

print(f'время выполнения лемматизации с POS-тегами: {data_lemm_pos_time} s')

время выполнения лемматизации с POS-тегами: 3270 s
CPU times: total: 50min 43s
Wall time: 54min 30s


In [17]:
# промежуточно сохраняем файл с очищенными и лемматизированными комментариями
'''
data.to_csv('data_lemm2.csv', index=False)
data = pd.read_csv('data_lemm2.csv')'''

"\ndata.to_csv('data_lemm2.csv', index=False)\ndata = pd.read_csv('data_lemm2.csv')"

Выведем полученный датасет `data` с новыми признаками `wnl_text` и `wnlpostag_text`.

In [18]:
display(data.sample(3).T)

,151541,20160,116955
Unnamed: 0,151697,20180,117054
text,Something must be done!,Well thanks for all your Bundeswehr Heer addit...,"""\n\nI removed it because it was unsourced. I ..."
toxic,0,0,0
clean_text,something must done,well thanks bundeswehr heer additions division...,removed unsourced think article neutral point ...
wnl_text,something must done,well thanks bundeswehr heer addition division ...,removed unsourced think article neutral point ...
wnlpostag_text,something must do,well thanks bundeswehr heer addition division ...,remove unsourced think article neutral point v...


<a name="35"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 3.5. Вывод

В разделе [**Подготовка данных**](#3.-Подготовка-данных) были выполнены следующие задачи:
1. данные изучены;
2. комментарии очищены: буквы переведены в нижний регистр, оставлена только латиница, стоп-слова удалены; 
3. комментарии лемматизированы без учёта части речи (без *POS*-тегов);
4. комментарии лемматизированы с учётом части речи (с *POS*-тегами).


В результате выполнения задач этого раздела было выявлено следующее:
1. пропусков в данных нет;
2. типы данных соответствуют требованиям для последующей очистки и лемматизации комментариев;
3. датасет имеет большой размер: содержит 159 292 текстовых комментария;
4. явных дубликатов нет;
5. наблюдается сильный дисбаланс классов в таргете.

**В проекте решается задача бинарной классификации.**

Таким образом, данные подготовлены для обучения моделей.

<a name="4"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
## 4. Обучение моделей

<a name="41"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.1. Разделение данных на выборки

Зададим параметры для кроссвалидации: `n_splits` - количество фолдов.

In [19]:
kfold = KFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True)

Разделим данные на обучающую и тестовую выборки для обоих признаков `wnl_text` и `wnlpostag_text` в соотношении 4:1.

In [20]:
features_train, features_test, target_train, target_test = train_test_split(
    data['wnl_text'], data['toxic'].values, test_size=TEST_SIZE, stratify=data['toxic'].values, 
    shuffle=True, random_state=RANDOM_STATE)

In [21]:
features_train_pos, features_test_pos, target_train_pos, target_test_pos = train_test_split(
    data['wnlpostag_text'], data['toxic'].values, test_size=TEST_SIZE, stratify=data['toxic'].values, 
    shuffle=True, random_state=RANDOM_STATE)

Оценка важности слова определяется величиной *TF-IDF* (от англ. *term frequency*, «частота терма, или слова»; *inverse document frequency*, «обратная частота документа, или текста»). То есть *TF* отвечает за количество упоминаний слова в отдельном тексте, а *IDF* отражает частоту его употребления во всём корпусе.

Рассчитаем *TF-IDF* для обоих признаков `wnl_text` и `wnlpostag_text`.

In [22]:
count_tf_idf = TfidfVectorizer()
tf_idf_train = count_tf_idf.fit_transform(features_train)
tf_idf_test = count_tf_idf.transform(features_test)

In [23]:
count_tf_idf_pos = TfidfVectorizer()
tf_idf_train_pos = count_tf_idf_pos.fit_transform(features_train_pos)
tf_idf_test_pos = count_tf_idf_pos.transform(features_test_pos)

Для обоих признаков `wnl_text` и `wnlpostag_text`обучим восемь моделей (`LogisticRegression`, `DecisionTreeClassifier` и `LGBMClassifier` с количеством деревьев `n_estimators`=50 и `n_estimators`=500). Для обучения моделей используем функцию `cross_val_score()`. Учтём баланс классов в модели с помощью параметра `class_weight`.

<a name="42"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.2. `LogisticRegression`

Построим модель **логистической регрессии *Logistic Regression*** для обоих признаков `wnl_text` и `wnlpostag_text`.

<a name="421"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
#### 4.2.1. `model_lr`

Построим модель **логистической регрессии *Logistic Regression*** для признака `wnl_text` (без *POS*-тегов).

In [24]:
%%time

beg_time = datetime.datetime.now()

model_lr = LogisticRegression(solver='liblinear', 
                              class_weight='balanced', 
                              random_state=RANDOM_STATE)

model_lr.mod = 'model_lr'
model_lr.name = 'LogisticRegression'
model_lr.data = 'wnl_text'
model_lr.f1 = cross_val_score(model_lr,
                              tf_idf_train, 
                              target_train, 
                              cv=kfold, 
                              scoring='f1')

model_lr.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_lr.f1.mean()))
print('модель:', model_lr.name)
print('данные:', model_lr.data)
print(f'время работы модели: {model_lr.time} s')

f1: 0.753
модель: LogisticRegression
данные: wnl_text
время работы модели: 5 s
CPU times: total: 20.7 s
Wall time: 5.28 s


- метрика `f1`: 0.753
- модель: `LogisticRegression` 
- данные: `wnl_text`
- время работы модели: 5 s

<a name="422"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
#### 4.2.2. `model_lr_pos`

Построим модель **логистической регрессии *Logistic Regression*** для признака `wnlpostag_text` (с *POS*-тегами).

In [25]:
%%time

beg_time = datetime.datetime.now()

model_lr_pos = LogisticRegression(solver='liblinear', 
                                  class_weight='balanced', 
                                  random_state=RANDOM_STATE)

model_lr_pos.mod = 'model_lr_pos'
model_lr_pos.name = 'LogisticRegression'
model_lr_pos.data = 'wnlpostag_text'
model_lr_pos.f1 = cross_val_score(model_lr_pos, 
                                  tf_idf_train_pos, 
                                  target_train_pos, 
                                  cv=kfold, 
                                  scoring='f1')

model_lr_pos.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_lr_pos.f1.mean()))
print('модель:', model_lr_pos.name)
print('данные:', model_lr_pos.data)
print(f'время работы модели: {model_lr_pos.time} s')

f1: 0.752
модель: LogisticRegression
данные: wnlpostag_text
время работы модели: 5 s
CPU times: total: 21.6 s
Wall time: 5.45 s


- метрика `f1`: 0.752
- модель: `LogisticRegression` 
- данные: `wnlpostag_text` 
- время работы модели: 5 s

<a name="43"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.3. `DecisionTreeClassifier`

Построим модель **дерева решений *Decision Tree*** для обоих признаков `wnl_text` и `wnlpostag_text`.

<a name="431"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
#### 4.3.1. `model_dt`

Построим модель **дерева решений *Decision Tree*** для признака `wnl_text` (без *POS*-тегов).

In [26]:
%%time

beg_time = datetime.datetime.now()

model_dt = DecisionTreeClassifier(max_depth=15, 
                                  class_weight='balanced', 
                                  random_state=RANDOM_STATE)

model_dt.mod = 'model_dt'
model_dt.name = 'DecisionTreeClassifier'
model_dt.data = 'wnl_text'
model_dt.f1 = cross_val_score(model_dt, 
                              tf_idf_train, 
                              target_train, 
                              cv=kfold, 
                              scoring='f1')

model_dt.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_dt.f1.mean()))
print('модель:', model_dt.name)
print('данные:', model_dt.data)
print(f'время работы модели: {model_dt.time} s')

f1: 0.618
модель: DecisionTreeClassifier
данные: wnl_text
время работы модели: 85 s
CPU times: total: 1min 25s
Wall time: 1min 25s


- метрика `f1`: 0.618
- модель: `DecisionTreeClassifier`
- глубина дерева: `max_depth` = 15
- данные: `wnl_text`
- время работы модели: 85 s

<a name="432"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
#### 4.3.2. `model_dt_pos`

Построим модель **дерева решений *Decision Tree*** для признака `wnlpostag_text` (с *POS*-тегами).

In [27]:
%%time

beg_time = datetime.datetime.now()

model_dt_pos = DecisionTreeClassifier(max_depth=15,
                                      class_weight='balanced', 
                                      random_state=RANDOM_STATE)

model_dt_pos.mod = 'model_dt_pos'
model_dt_pos.name = 'DecisionTreeClassifier'
model_dt_pos.data = 'wnlpostag_text'
model_dt_pos.f1 = cross_val_score(model_dt_pos, 
                                  tf_idf_train_pos, 
                                  target_train_pos, 
                                  cv=kfold, 
                                  scoring='f1')

model_dt_pos.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_dt_pos.f1.mean()))
print('модель:', model_dt_pos.name)
print('данные:', model_dt_pos.data)
print(f'время работы модели: {model_dt_pos.time} s')

f1: 0.625
модель: DecisionTreeClassifier
данные: wnlpostag_text
время работы модели: 85 s
CPU times: total: 1min 25s
Wall time: 1min 25s


- метрика `f1`: 0.625
- модель: `DecisionTreeClassifier`
- глубина дерева: `max_depth` = 15
- данные: `wnlpostag_text` 
- время работы модели: 85 s

<a name="44"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.4. `LGBMClassifier`

Построим модель **градиентного бустинга *LightGBM*** с количеством деревьев `n_estimators`=50 и `n_estimators`=500 для обоих признаков `wnl_text` и `wnlpostag_text`.

<a name="441"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
#### 4.4.1. `model_lgbm50`

Построим модель **градиентного бустинга *LightGBM*** с количеством деревьев `n_estimators`=50 для признака `wnl_text` (без *POS*-тегов).

In [28]:
%%time

beg_time = datetime.datetime.now()

model_lgbm50 = LGBMClassifier(n_estimators=50,
                              max_depth=10,
                              learning_rate=0.15,
                              class_weight='balanced', 
                              boosting_type='gbdt', 
                              objective='binary', 
                              random_state=RANDOM_STATE)

model_lgbm50.mod = 'model_lgbm50'
model_lgbm50.name = 'LGBMClassifier 50'
model_lgbm50.data = 'wnl_text'
model_lgbm50.f1 = cross_val_score(model_lgbm50, 
                                  tf_idf_train, 
                                  target_train, 
                                  cv=kfold, 
                                  scoring='f1')

model_lgbm50.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_lgbm50.f1.mean()))
print('модель:', model_lgbm50.name)
print('данные:', model_lgbm50.data)
print(f'время работы модели: {model_lgbm50.time} s')

f1: 0.724
модель: LGBMClassifier 50
данные: wnl_text
время работы модели: 42 s
CPU times: total: 4min 23s
Wall time: 42.5 s


- метрика `f1`: 0.724
- модель: `LGBMClassifier`
- количество деревьев: `n_estimators` = 50
- глубина дерева: `max_depth` = 10
- коэффициент скорости обучения: `learning_rate` = 0.15
- данные: `wnl_text`  
- время работы модели: 42 s

<a name="442"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
#### 4.4.2. `model_lgbm50_pos`

Построим модель **градиентного бустинга *LightGBM*** с количеством деревьев `n_estimators`=50 для признака `wnlpostag_text` (с *POS*-тегами).

In [29]:
%%time

beg_time = datetime.datetime.now()

model_lgbm50_pos = LGBMClassifier(n_estimators=50,
                                  max_depth=10,
                                  learning_rate=0.15,
                                  class_weight='balanced', 
                                  boosting_type='gbdt', 
                                  objective='binary', 
                                  random_state=RANDOM_STATE)

model_lgbm50_pos.mod = 'model_lgbm50_pos'
model_lgbm50_pos.name = 'LGBMClassifier 50'
model_lgbm50_pos.data = 'wnlpostag_text'
model_lgbm50_pos.f1 = cross_val_score(model_lgbm50_pos, 
                                      tf_idf_train_pos, 
                                      target_train_pos, 
                                      cv=kfold, 
                                      scoring='f1')

model_lgbm50_pos.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_lgbm50_pos.f1.mean()))
print('модель:', model_lgbm50_pos.name)
print('данные:', model_lgbm50_pos.data)
print(f'время работы модели: {model_lgbm50_pos.time} s')

f1: 0.730
модель: LGBMClassifier 50
данные: wnlpostag_text
время работы модели: 40 s
CPU times: total: 4min 2s
Wall time: 40.9 s


- метрика `f1`: 0.730
- модель: `LGBMClassifier` 
- количество деревьев: `n_estimators` = 50
- глубина дерева: `max_depth` = 10
- коэффициент скорости обучения: `learning_rate` = 0.15
- данные: `wnlpostag_text`  
- время работы модели: 40 s

<a name="443"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
#### 4.4.3. `model_lgbm500`

Построим модель **градиентного бустинга *LightGBM*** с количеством деревьев `n_estimators`=500 для признака `wnl_text` (без *POS*-тегов).

In [30]:
%%time

beg_time = datetime.datetime.now()

model_lgbm500 = LGBMClassifier(n_estimators=500,
                               max_depth=10,
                               learning_rate=0.15,
                               class_weight='balanced', 
                               boosting_type='gbdt', 
                               objective='binary', 
                               random_state=RANDOM_STATE)

model_lgbm500.mod = 'model_lgbm500'
model_lgbm500.name = 'LGBMClassifier 500'
model_lgbm500.data = 'wnl_text'
model_lgbm500.f1 = cross_val_score(model_lgbm500, 
                                   tf_idf_train, 
                                   target_train, 
                                   cv=kfold, 
                                   scoring='f1')

model_lgbm500.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_lgbm500.f1.mean()))
print('модель:', model_lgbm500.name)
print('данные:', model_lgbm500.data)
print(f'время работы модели: {model_lgbm500.time} s')

f1: 0.771
модель: LGBMClassifier 500
данные: wnl_text
время работы модели: 179 s
CPU times: total: 19min
Wall time: 2min 59s


- метрика `f1`: 0.771
- модель: `LGBMClassifier` 
- количество деревьев: `n_estimators` = 500
- глубина дерева: `max_depth` = 10
- коэффициент скорости обучения: `learning_rate` = 0.15
- данные: `wnl_text`  
- время работы модели: 179 s

<a name="444"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
#### 4.4.4. `model_lgbm500_pos`

Построим модель **градиентного бустинга *LightGBM*** с количеством деревьев `n_estimators`=500 для признака `wnlpostag_text` (с *POS*-тегами).

In [31]:
%%time

beg_time = datetime.datetime.now()

model_lgbm500_pos = LGBMClassifier(n_estimators=500,
                                   max_depth=10,
                                   learning_rate=0.15,
                                   class_weight='balanced', 
                                   boosting_type='gbdt', 
                                   objective='binary', 
                                   random_state=RANDOM_STATE)

model_lgbm500_pos.mod = 'model_lgbm500_pos'
model_lgbm500_pos.name = 'LGBMClassifier 500'
model_lgbm500_pos.data = 'wnlpostag_text'
model_lgbm500_pos.f1 = cross_val_score(model_lgbm500_pos, 
                                       tf_idf_train_pos, 
                                       target_train_pos, 
                                       cv=kfold, 
                                       scoring='f1')

model_lgbm500_pos.time = (datetime.datetime.now()-beg_time).seconds

print('f1: %.3f' %(model_lgbm500_pos.f1.mean()))
print('модель:', model_lgbm500_pos.name)
print('данные:', model_lgbm500_pos.data)
print(f'время работы модели: {model_lgbm500_pos.time} s')

f1: 0.772
модель: LGBMClassifier 500
данные: wnlpostag_text
время работы модели: 174 s
CPU times: total: 17min 40s
Wall time: 2min 54s


- метрика `f1`: 0.772
- модель: `LGBMClassifier` 
- количество деревьев: `n_estimators` = 500
- глубина дерева: `max_depth` = 10
- коэффициент скорости обучения: `learning_rate` = 0.15
- данные: `wnlpostag_text`  
- время работы модели: 174 s

<a name="45"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.5. Сравнение моделей

Сравним модели и выберем лучшую, т.е. с наибольшим значением метрики *F1*.

In [32]:
model_list = [model_lr, model_lr_pos, model_dt, model_dt_pos, 
              model_lgbm50, model_lgbm50_pos, model_lgbm500, model_lgbm500_pos]

In [33]:
a={}
for i in model_list:
    b={}    
    b['model']=i.name
    b['data']=i.data
    b['f1_score']=i.f1.mean()
    b['cross_val_time']=i.time
    a[i.mod] = b

final_table = pd.DataFrame(a)

display(final_table.T)

,model,data,f1_score,cross_val_time
model_lr,LogisticRegression,wnl_text,0.752913,5
model_lr_pos,LogisticRegression,wnlpostag_text,0.752096,5
model_dt,DecisionTreeClassifier,wnl_text,0.61783,85
model_dt_pos,DecisionTreeClassifier,wnlpostag_text,0.624639,85
model_lgbm50,LGBMClassifier 50,wnl_text,0.724338,42
model_lgbm50_pos,LGBMClassifier 50,wnlpostag_text,0.730063,40
model_lgbm500,LGBMClassifier 500,wnl_text,0.770725,179
model_lgbm500_pos,LGBMClassifier 500,wnlpostag_text,0.771799,174


Выберем в качестве лучшей модель **градиентного бустинга `LGBMClassifier`** `model_lgbm500_pos`, которая имеет следующее значение метрики оценки качества **на обучающей выборке**:

- метрика `f1`: 0.772
- модель: `LGBMClassifier` 
- количество деревьев: `n_estimators` = 500
- глубина дерева: `max_depth` = 10
- коэффициент скорости обучения: `learning_rate` = 0.15
- данные: `wnlpostag_text`  
- время работы модели: 174 s

<a name="46"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 4.6. Вывод

В разделе [**Обучение моделей**](#4.-Обучение-моделей) были выполнены следующие задачи:
1. Для обоих признаков `wnl_text` и `wnlpostag_text` данные разделены на обучающую и тестовую выборки.
2. Для обоих признаков `wnl_text` и `wnlpostag_text` рассчитана величина *TF-IDF*.
3. Для обоих признаков `wnl_text` и `wnlpostag_text`обучены восемь моделей (`LogisticRegression`, `DecisionTreeClassifier` и `LGBMClassifier` с количеством деревьев `n_estimators`=50 и `n_estimators`=500).
4. Выведена таблица сравнения моделей.


В результате выполнения задач этого раздела было выявлено следующее:
- в качестве лучшей (с наибольшим значением метрики *F1*) выбрана модель **градиентного бустинга `LGBMClassifier`** `model_lgbm500_pos`, которая имеет следующее значение метрики оценки качества **на обучающей выборке**:

    - метрика `f1`: 0.772
    - модель: `LGBMClassifier` 
    - количество деревьев: `n_estimators` = 500
    - глубина дерева: `max_depth` = 10
    - коэффициент скорости обучения: `learning_rate` = 0.15
    - данные: `wnlpostag_text`  
    - время работы модели: 174 s

<a name="5"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
## 5. Тестирование лучшей модели

<a name="51"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 5.1. Качество модели

Проверим лучшую модель **градиентного бустинга `LGBMClassifier`** `model_lgbm500_pos` **на тестовой выборке**.

In [34]:
%%time

model_lgbm500_pos.fit(tf_idf_train_pos, target_train_pos)
model_lgbm500_pos.predicted = model_lgbm500_pos.predict(tf_idf_test_pos)
model_lgbm500_pos.test_f1 = f1_score(target_test_pos, model_lgbm500_pos.predicted)
print('f1: %.3f' %(model_lgbm500_pos.test_f1))

f1: 0.775
CPU times: total: 4min 21s
Wall time: 43.7 s


- метрика `f1`: 0.775

<a name="52"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
### 5.2. Вывод

В разделе [**Тестирование лучшей модели**](#5.-Тестирование-лучшей-модели) была протестирована лучшая модель градиентного бустинга `LGBMClassifier` `model_lgbm500_pos` с количеством деревьев `n_estimators`=500 для признака `wnlpostag_text` (с *POS*-тегами).
   
В результате выполнения задач этого раздела было выявлено следующее:
- для выбранной лучшей модели значение метрики качества ***F1 = 0.775***, что превышает 0.75, как и изначально требовалось по условию задачи проекта.

<a name="6"></a> <div style="text-align: right">[Cодержание](#1.-Содержание)</div>
## 6. Общий вывод

Проведено исследование с целью построения модели машинного обучения, которая поможет классифицировать комментарии на позитивные и негативные.

Результаты исследования позволят магазину искать токсичные комментарии и отправлять их на модерацию.

Входные данные: набор данных с разметкой о токсичности правок.

В ходе исследования удалось получить следующие результаты **на обучающей выборке**:


1. **Модель `LogisticRegression`**:

   - данные, лемматизированые без учёта части речи - без *POS*-тегов (для признака `wnl_text`):
      - *F1* = 0.753
      - время работы модели: 5 s
   - данные, лемматизированые с учётом части речи - с *POS*-тегами (для признака `wnlpostag_text`):
      - *F1* = 0.752
      - время работы модели: 5 s


2. **Модель `DecisionTreeClassifier`** с глубиной дерева `max_depth` = 15:

   - данные, лемматизированые без учёта части речи - без *POS*-тегов (для признака `wnl_text`):
      - *F1* = 0.618
      - время работы модели: 85 s
   - данные, лемматизированые с учётом части речи - с *POS*-тегами (для признака `wnlpostag_text`):
      - *F1* = 0.625
      - время работы модели: 85 s


3. **Модель `LGBMClassifier`** с глубиной дерева `max_depth` = 10 и коэффициентом скорости обучения `learning_rate` = 0.15:

- количество деревьев **`n_estimators` = 50**:

   - данные, лемматизированые без учёта части речи - без *POS*-тегов (для признака `wnl_text`):
      - *F1* = 0.724
      - время работы модели: 42 s
   - данные, лемматизированые с учётом части речи - с *POS*-тегами (для признака `wnlpostag_text`):
      - *F1* = 0.730
      - время работы модели: 40 s


- количество деревьев **`n_estimators` = 500**:

   - данные, лемматизированые без учёта части речи - без *POS*-тегов (для признака `wnl_text`):
      - *F1* = 0.771
      - время работы модели: 179 s
   - данные, лемматизированые с учётом части речи - с *POS*-тегами (для признака `wnlpostag_text`):
      - *F1* = 0.772
      - время работы модели: 174 s
            
      
Исходя из полученных результатов, можно сделать следующие **выводы**:


1. В качестве лучшей модели выбрана:
   - модель **градиентного бустинга `LGBMClassifier`** `model_lgbm500_pos`, которая **на тестовой выборке** имеет следующее значение метрики оценки качества:
    - ***F1 = 0.775*** 
   
   
2. Для выбранной лучшей модели значение метрики качества *F1* превышает 0.75, что соответсвует изначальному требованию в условии задачи проекта.
   
   
**Общие рекомендации:**

Магазину можно рекомендовать использовать полученную модель **`LGBMClassifier`** в качестве инструмента, который будет искать токсичные комментарии и отправлять их на модерацию.